In [1]:
import json
import pandas as pd
from copy import deepcopy

In [2]:
%%capture --no-stderr
%pip install -U langchain-nomic langchain_community tiktoken langchainhub chromadb langchain langgraph nomic[local]

## Load Data

In [3]:
# load data

try:
    with  open('data/mails.json', 'r') as f:
        email_database = json.load(f)
except FileNotFoundError:
    raise FileNotFoundError('mails.json not found')

print(f"Il y a {len(email_database)} mails (question+réponse) dans la base de données")

try :
    actis = pd.read_csv('data/actis.csv')
except FileNotFoundError:
    raise FileNotFoundError('actis.csv not found')

print(f"Il y a {len(actis)} actis dans la base de données")

try :
    chambres = pd.read_csv('data/chambres.csv')
except FileNotFoundError:
    raise FileNotFoundError('chambres.csv not found')

print(f"Il y a {len(chambres)} semaines de dispos renseignées pour les chambres dans la base de données")

try :
    accueil = pd.read_csv('data/accueil.csv')
except FileNotFoundError:
    raise FileNotFoundError('accueil.csv not found')

print(f"Il y a {len(accueil)} informations à propos de l'accueil dans la base de données")

Il y a 49 mails (question+réponse) dans la base de données
Il y a 30 actis dans la base de données
Il y a 52 semaines de dispos renseignées pour les chambres dans la base de données
Il y a 7 informations à propos de l'accueil dans la base de données


In [4]:
#local_llm = "llama3.1"
local_llm = "gemma2:27b"
local_embedder = "nomic-embed-text"

In [5]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama

embedder = OllamaEmbeddings(model = local_embedder)
llm = ChatOllama(model = local_llm, temperature=0)

## Choose where to fecth the information from

In [6]:
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda

In [7]:
from ollama_instructor.ollama_instructor_client import OllamaInstructorClient

In [8]:
from datetime import datetime

In [9]:
#2023-02-19
datetime.strptime("2023-02-19", "%Y-%m-%d")

datetime.datetime(2023, 2, 19, 0, 0)

In [10]:
## gérer les mails

def try_content_from_mail(mail):
    from langchain_core.documents import Document
    with  open('data/mails.json', 'r') as f:
            email_database = json.load(f)

    full_text = []
    for i, page in enumerate(email_database):
        question = page['question']
        answer = page['answer']
        document = Document(
            page_content=f" QUESTION:\n {question}\n\n ANSWER:\n {answer}",
        )
        full_text.append(document)

    from langchain_chroma import Chroma
    embedder = OllamaEmbeddings(model = local_embedder)

    vectorstore = Chroma()
    vectorstore.delete_collection()
    vectorstore = Chroma.from_documents(documents=full_text, 
                                        embedding=embedder)
    retriever = vectorstore.as_retriever()

    TEMPLATE = """Tu es un assistant qui va m'aider à répondre à des mails. J'ai déjà une base énorme de mails avec des questons et de réponses, et je veux que tu m'aides à répondre à des questions.
    Je vaius te donner en contexte une paire question/réponse parmi mes mails, puis une question. Tu devras me donner la réponse qui correspond le mieux à la question. Tu peux dire "je ne sais pas" si tu ne sais pas.

    Voici un exemple de question et de réponse :
    {context}

    Voici la question à laquelle tu dois répondre:
    {question}

    Tu vas UNIQUEMENT me donner la réponse à donner à la question. Tu n'as pas besoin de me donner la question, je la connais déjà.
    """

    prompt = ChatPromptTemplate.from_template(TEMPLATE)
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
    )

    res = rag_chain.invoke(mail)
    return res


In [11]:
class RouteQuery(BaseModel):
    """Tu vas guider l'utilisateur vers la bonne source de données pour répondre à sa question."""

    datasource: Literal["disponibilite_chambres", "ouverture_accueil", "activite", "other"] = Field(
        ...,
        description="On te donne une question d'un utilisateur, et tu dois choisir la source de données la plus pertinente pour répondre à sa question. \
            disponibilite_chambres se réfère à la disponibilité des chambres. \
            ouverture_accueil se réfère aux horaires d'ouverture de l'accueil. \
            activites se réfère aux différentes activités de loisir proposées par l'hotel. \
            other se réfère à une question qui ne rentre pas dans les autres catégories."
    )

class RoomDates(BaseModel):
    """Gives the start and end dates of a room reservation that a user is asking about, in 2023, as well as the type of room."""

    start_date: str = Field(
        ...,
        description="The start date of the room reservation that the user is asking about.",
    )
    end_date: str = Field(
        ...,
        description="The end date of the room reservation that the user is asking about.",
    )
    room_type: Literal["double", "family", "not_specified"] = Field(
        ...,
        description="The type of room that the user is asking about."
    )

client = OllamaInstructorClient()

def dispo_chambre(mail: str) -> RoomDates:
    response = client.chat_completion(
        model='llama3.1', 
        pydantic_model=RoomDates, 
        messages=[
            {
                'role': 'user',
                'content': mail
            }
        ]
    )
    start_date = response["message"]["content"]["start_date"]
    end_date = response["message"]["content"]["end_date"]
    room_type = response["message"]["content"]["room_type"]

    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    data = pd.read_csv('data/chambres.csv')

    week_beg = None
    week_end = None

    for index, row in data.iterrows():
        if datetime.strptime(row['Date_Debut'], "%Y-%m-%d") >= start_date and week_beg is None:
            week_beg = index - 1
        if datetime.strptime(row['Date_Fin'], "%Y-%m-%d") >= end_date:
            week_end = index
        if week_beg is not None and week_end is not None:
            break    
    
    if week_beg is None or week_end is None:
        raise Exception("No data found for this period")

    data = data.iloc[week_beg:week_end]
    return {"start_date": start_date, "end_date": end_date, "room_type": room_type, "data": data}

def choose_route(mail: str) -> RouteQuery:
    response = client.chat_completion(
        model='llama3.1', 
        pydantic_model=RouteQuery, 
        messages=[
            {
                'role': 'user',
                'content': mail
            }
        ]
    )
    if response is None:
        return {"question": mail, "datasource": "other"}
    return {"question": mail, "datasource": response["message"]["content"]['datasource']}

def continue_root(state):
    question = state["question"]
    datasource = state["datasource"]
    print('-----------------')
    print(f"Question : {question}")
    print(f"Datasource : {datasource}")
    print('-----------------')
    if datasource == "ouverture_accueil":
        data = pd.read_csv('data/accueil.csv')
        PROMPT = f"""Tu es un assistant va répondre à une question de l'utilisateur à propos des horaires d'ouverture de l'accueil. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

        Question : {question}

        
        Tu vas répondre à l'aide de ces données:
        Données :
        """ + data.to_string()
        return PROMPT
    
    elif datasource == "disponibilite_chambres":
        out = dispo_chambre(question)
        PROMPT = f"""Tu es un assistant va répondre à une question de l'utilisateur à propos de la disponibilité des chambres. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

        Question : {question}

        Tu vas prendre en compte la période suivante : {out["start_date"]} - {out["end_date"]} pour une chambre de type {out["room_type"]}.

        Tu vas répondre à l'aide de ces données:
        Données :
        """ + out["data"].to_string()
        return PROMPT
    
    elif datasource == "activite":
        data = pd.read_csv('data/actis.csv')
        PROMPT = f"""Tu es un assistant va répondre à une question de l'utilisateur à propos des prix des activités. Tu vas être très simple et factuel. Tu vas t'exprimer clairement.

        Question : {question}

        Tu vas répondre à l'aide de ces données.
        Données :
        """ + data.to_string()
        return PROMPT
    
    elif datasource == "other":
        return try_content_from_mail(question)
    

first_classifier = RunnableLambda(choose_route)  

In [12]:
from tqdm import tqdm
# First classifier test

#1 - Disponibilité des chambres

def isRoom(state):
    return state["datasource"] == "disponibilite_chambres"

test_rooms = first_classifier | RunnableLambda(isRoom)

queries = [
    "Avez-vous des chambres disponibles pour ce week-end ?",
    "Quelles sont les disponibilités de vos chambres pour le mois prochain ?",
    "Je voudrais réserver une chambre, est-ce qu'il y en a de libres ?",
    "Y a-t-il des chambres libres pour deux personnes demain soir ?",
    "Je cherche une chambre simple pour trois nuits, avez-vous de la disponibilité ?",
    "Est-ce qu'il reste des chambres pour le 25 septembre ?",
    "Pouvez-vous me dire si vous avez des chambres pour la semaine prochaine ?",
    "Je souhaite réserver une chambre familiale, est-ce qu'il en reste ?",
    "Quelles sont vos disponibilités pour une chambre avec vue sur la mer ?",
    "Est-ce qu'il reste des suites disponibles pour ce soir ?",
    "J'aimerais savoir si vous avez des chambres pour quatre personnes le 10 octobre.",
    "Je voudrais réserver une chambre avec un grand lit, est-ce possible pour demain ?",
    "Y a-t-il des chambres disponibles pour une personne du 15 au 18 novembre ?",
    "Je voudrais savoir si vous avez des chambres doubles disponibles pour la semaine prochaine.",
    "Avez-vous encore des chambres pour deux nuits à partir de vendredi ?",
    "Je cherche une chambre avec balcon, est-ce qu'il en reste pour ce week-end ?",
    "Pouvez-vous vérifier si vous avez des chambres libres du 1er au 3 décembre ?",
    "Je souhaite réserver une chambre pour trois personnes le 22 septembre, est-ce possible ?",
    "Est-ce qu'il y a des chambres libres pour ce soir ?",
    "Je voudrais savoir si vous avez des chambres pour quatre nuits à partir de jeudi.",
    "Je cherche une chambre avec baignoire, avez-vous de la disponibilité ?",
    "Pouvez-vous me dire si vous avez des suites disponibles pour le mois prochain ?",
    "Je souhaite réserver une chambre double, est-ce qu'il y en a encore ?",
    "Quelles sont vos disponibilités pour une chambre simple en novembre ?",
    "Est-ce qu'il reste des chambres avec petit-déjeuner inclus pour ce week-end ?",
    "Y a-t-il des chambres disponibles pour deux adultes et un enfant ce samedi ?",
    "Je voudrais savoir si vous avez des chambres de luxe pour cette semaine.",
    "Avez-vous des chambres avec vue sur le jardin de disponibles pour septembre ?",
    "Je cherche une chambre économique, est-ce qu'il en reste pour demain soir ?",
    "Est-ce que je peux encore réserver une chambre pour trois nuits la semaine prochaine ?"
]

count = 0
for query in tqdm(queries):
    if test_rooms.invoke(query):
        count += 1
print(f"{count}/{len(queries)} questions ont été correctement classifiées comme étant des questions sur la disponibilité des chambres")

100%|██████████| 30/30 [02:10<00:00,  4.34s/it]

27/30 questions ont été correctement classifiées comme étant des questions sur la disponibilité des chambres


In [13]:
# 2 - Ouverture de l'accueil

def isAccueil(state):
    return state["datasource"] == "ouverture_accueil"

test_accueil = first_classifier | RunnableLambda(isAccueil)

queries = [
    "Quels sont les horaires d'ouverture de la réception ?",
    "Est-ce que l'accueil est ouvert toute la nuit ?",
    "Pouvez-vous me dire jusqu'à quelle heure la réception est ouverte ?",
    "À quelle heure ouvre l'accueil demain matin ?",
    "Est-ce que je peux arriver tard, et la réception sera encore ouverte ?",
    "L'accueil ferme à quelle heure ce soir ?",
    "À quelle heure puis-je faire mon check-in ?",
    "Est-ce que la réception est ouverte 24h/24 ?",
    "Quand est-ce que la réception ferme aujourd'hui ?",
    "L'accueil est-il encore ouvert à 23h ?",
    "J'arrive tard, à quelle heure ferme la réception ?",
    "Quels sont vos horaires d'accueil ?",
    "La réception est-elle ouverte le matin ?",
    "Je voudrais savoir quand je peux venir à la réception.",
    "Pouvez-vous me dire si l'accueil est ouvert après 22h ?",
    "L'accueil est-il ouvert le dimanche ?",
    "Est-ce que je peux venir à la réception à 7h du matin ?",
    "Quels sont vos horaires de réception en semaine ?",
    "À quelle heure la réception ferme-t-elle le vendredi ?",
    "Est-ce que la réception est ouverte pendant les jours fériés ?",
    "Je voudrais savoir à quelle heure je peux faire mon check-out à la réception.",
    "L'accueil sera-t-il ouvert si j'arrive tard le soir ?",
    "Pouvez-vous me confirmer les horaires d'ouverture de l'accueil pour demain ?",
    "À quelle heure je peux venir récupérer mes clés à la réception ?",
    "L'accueil ferme-t-il à 21h ce soir ?",
    "Est-ce que l'accueil est disponible à partir de 6h le matin ?",
    "À quelle heure la réception ouvre-t-elle le samedi ?",
    "Est-ce que la réception est ouverte à partir de 8h demain ?",
    "Quels sont les horaires de l'accueil pendant le week-end ?",
    "Je voudrais savoir si l'accueil ferme plus tôt les jours fériés."
]

count = 0
for query in tqdm(queries):
    if test_accueil.invoke(query):
        count += 1
print(f"{count}/{len(queries)} questions ont été correctement classifiées comme étant des questions sur l'ouverture de l'accueil")

100%|██████████| 30/30 [03:13<00:00,  6.44s/it]

26/30 questions ont été correctement classifiées comme étant des questions sur l'ouverture de l'accueil


In [14]:
# 3 - Activités

def isActivite(state):
    return state["datasource"] == "activite"

test_activite = first_classifier | RunnableLambda(isActivite)

queries = [
    "Quelles activités proposez-vous pendant les vacances ?",
    "Avez-vous des cours de yoga à l'hôtel ?",
    "Est-ce que vous proposez des sorties en voilier ?",
    "Je voudrais savoir si vous avez des activités comme le masque et tuba.",
    "Y a-t-il des excursions organisées depuis l'hôtel ?",
    "Pouvez-vous me dire si vous proposez des cours de yoga et à quel prix ?",
    "Avez-vous des activités nautiques pour les clients de l'hôtel ?",
    "Est-ce qu'il y a des sorties en bateau proposées pendant mon séjour ?",
    "Proposez-vous des activités de groupe comme le beach-volley ?",
    "Je cherche des activités relaxantes comme le yoga, en proposez-vous ?",
    "Quels sont les prix pour les cours de voile ?",
    "Est-ce que l'hôtel organise des sessions de snorkeling ?",
    "Y a-t-il des activités de plage comme le paddle ou le kayak ?",
    "Pouvez-vous me dire si vous proposez des cours de méditation ?",
    "Proposez-vous des activités pour les enfants pendant les vacances ?",
    "Quels sont les tarifs pour une excursion en voilier ?",
    "Je voudrais savoir s'il y a des cours de fitness ou de sport proposés.",
    "Avez-vous des activités sportives comme le tennis ou le volley-ball ?",
    "Est-ce qu'il y a des séances de yoga gratuites à l'hôtel ?",
    "Quels sont les prix pour une session de masque et tuba ?",
    "L'hôtel organise-t-il des randonnées ou balades guidées ?",
    "Pouvez-vous me dire s'il y a des activités en soirée, comme des spectacles ou des animations ?",
    "Est-ce que vous proposez des sorties pour faire de la plongée sous-marine ?",
    "Quels types d'activités de bien-être proposez-vous à l'hôtel ?",
    "Est-ce qu'il y a des cours de pilates durant la semaine ?",
    "Je voudrais savoir s'il est possible de louer des voiliers depuis l'hôtel.",
    "Quels sont les tarifs pour vos activités nautiques ?",
    "Y a-t-il des excursions en groupe organisées pour les clients de l'hôtel ?",
    "Proposez-vous des cours de surf ou de paddle pendant les vacances ?",
    "Pouvez-vous me dire quelles activités de plage sont disponibles et leurs prix ?"
]


count = 0
for query in tqdm(queries):
    if test_activite.invoke(query):
        count += 1
print(f"{count}/{len(queries)} questions ont été correctement classifiées comme étant des questions sur les activités")

100%|██████████| 30/30 [02:19<00:00,  4.65s/it]

16/30 questions ont été correctement classifiées comme étant des questions sur les activités


In [15]:
chain = first_classifier | RunnableLambda(continue_root) | llm | StrOutputParser()

In [16]:
#mail = "Je veux venir mardi à 13h à l'accueil, est-ce que c'est possible ?"
#print(chain.invoke(mail))

In [17]:
#mail = "Je veux venir le 15 aout avec ma famille pour une semaine, est-ce que vous avez des chambres disponibles ?"
#print(chain.invoke(mail))

In [18]:
#mail = "Est-ce que vous avez une activitée de soin du visage ?"
#print(chain.invoke(mail))

In [19]:
mail = "Avez-vous une piscine ?"
print(chain.invoke(mail))

-----------------
Question : Avez-vous une piscine ?
Datasource : other
-----------------
Oui, nous avons une piscine sur place.
